<a href="https://colab.research.google.com/github/pharringtonp19/business-analytics/blob/main/notebooks/regression/causality_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pharringtonp19/business-analytics.git

Cloning into 'business-analytics'...
remote: Enumerating objects: 1468, done.
remote: Counting objects: 100% (523/523), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 1468 (delta 440), reused 271 (delta 262), pack-reused 945 (from 1)
Receiving objects: 100% (1468/1468), 24.39 MiB | 9.68 MiB/s, done.
Resolving deltas: 100% (896/896), done.


### **Import Packages**

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from functools import partial
import numpy as np

### **Variables to Change**

In [6]:
dataset = 'brookline.csv' # 'lifeExpectancy_reduced.csv'

if dataset == 'brookline.csv':
  dep_var = 'price'
  causal_var = 'size'
  vars_to_drop = ['stName', 'stNumber']
  categorical_vars = ['buildingStyle']

elif dataset == 'lifeExpectancy_reduced.csv':
  dep_var = 'Life_expectancy'
  causal_var = 'Alcohol_consumption'
  vars_to_drop = ['Country']
  categorical_vars = ['Region']

### **Read In Data Set**

In [7]:
df = pd.read_csv(f'/content/business-analytics/datasets/{dataset}')
df.head()

,price,stNumber,stName,size,beacon,baseFloor,buildingStyle,elevators,rooms,bedrooms,fullBathrooms,halfBathrooms,garage
0,174000,150,PLEASANT ST,1060,0,4,MID-RISE,1,4,2,1,1,1.0
1,337000,7,LEVERETT ST,831,0,1,DECKER,0,4,2,1,0,0.0
2,850000,24,EUSTON ST,2246,0,1,ROW-END,0,10,6,3,0,0.0
3,516000,417,WASHINGTON ST,1574,0,2,LOW-RISE,0,6,3,2,0,0.0
4,145000,150,PLEASANT ST,669,0,4,MID-RISE,1,3,1,1,0,1.0


### **Classify Variables**

In [8]:
eligible_vars = df.columns.drop(vars_to_drop + [dep_var, causal_var]).tolist()

### **Dictionary Comprehension**

In [9]:
eligible_vars_rep = {var : var if var not in categorical_vars else f'C({var})' for var in eligible_vars}
eligible_vars_rep

{'beacon': 'beacon',
 'baseFloor': 'baseFloor',
 'buildingStyle': 'C(buildingStyle)',
 'elevators': 'elevators',
 'rooms': 'rooms',
 'bedrooms': 'bedrooms',
 'fullBathrooms': 'fullBathrooms',
 'halfBathrooms': 'halfBathrooms',
 'garage': 'garage'}

### **Create Function**

In [10]:
def get_pvalues(reg_formula, var_rep):
  updated_reg_formula = reg_formula + ' + ' + var_rep
  linear_model  = smf.ols(updated_reg_formula, df)
  results = linear_model.fit()
  pvalues_series = results.pvalues
  pvalues_df = pd.DataFrame(list(pvalues_series.items()), columns=['variable', 'pvalue'])
  pvalues_df['base_variable'] = pvalues_df['variable'].map(lambda x: x.split('[')[0] if 'C(' in x else x)
  condition = pvalues_df['variable'] != 'Intercept'
  pvalues_df = pvalues_df[condition]
  pvalues_df = pvalues_df.groupby('base_variable')['pvalue'].min()
  return pvalues_df[var_rep]

### **Check Relationships**

In [12]:
realatedy = np.array(list(map(partial(get_pvalues, dep_var + ' ~ '), eligible_vars_rep.values()))) <= 0.05
relatedx =  np.array(list(map(partial(get_pvalues, causal_var + ' ~ '), eligible_vars_rep.values()))) <= 0.05

analysis_df = pd.DataFrame({'RelatedY': realatedy,
                            'RelatedX': relatedx}, index=eligible_vars)

analysis_df

,RelatedY,RelatedX
beacon,False,True
baseFloor,True,True
buildingStyle,True,True
elevators,True,True
rooms,True,True
bedrooms,True,True
fullBathrooms,True,True
halfBathrooms,True,True
garage,True,True


### **Select Variables**

In [13]:
condition = analysis_df['RelatedY'] & analysis_df['RelatedX']
selected_vars = analysis_df[condition].index.sort_values()
print(selected_vars)

Index(['baseFloor', 'bedrooms', 'buildingStyle', 'elevators', 'fullBathrooms',
       'garage', 'halfBathrooms', 'rooms'],
      dtype='object')


### **Estimator**

In [14]:
def Estimate(reg_formula):
  linear_model = smf.ols(reg_formula, data=df)
  results = linear_model.fit()
  return results

### **Compare Linear Models**

In [15]:
reg_formula1 = dep_var + ' ~ ' + causal_var
reg_formula2 = reg_formula1
for var in selected_vars:
  reg_formula2 += ' + ' + eligible_vars_rep[var]
results1 = Estimate(reg_formula1)
results2 = Estimate(reg_formula2)

### **Results**

In [16]:
summary_col([results1, results2])

,price I,price II
Intercept,12934.1240,-179170.5326
,(9705.7124),(31824.1450)
size,407.4513,313.0605
,(7.1667),(14.1024)
C(buildingStyle)[T.CONVERTED-HOUSE],,152982.9590
,,(31902.5700)
C(buildingStyle)[T.DECKER],,161802.3930
,,(28112.5005)
C(buildingStyle)[T.DUPLEX],,93006.3504
,,(35234.6627)
